# **Memory**

The memory allows a Large Language Model (LLM) to remember previous interactions with the user. By default, LLMs are stateless — meaning each incoming query is processed independently of other interactions.

In [ ]:
%%capture
# update or install the necessary libraries
%pip install --upgrade langchain_classic langchain_community langchain_aws python-dotenv 

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION")

# **ConversationBufferMemory**

ConversationBufferMemory usage is straightforward. It simply keeps the entire conversation in the buffer memory up to the allowed max limit (e.g. 4096 for gpt-3.5-turbo, 8192 for gpt-4). The main downside, however, is the cost. Each request is sending the aggregation to the API. Since the charge is based on token usages, the cost can quickly add up, especially if you are sending requests to an AI platform. Additionally, there can be added latency given the sheer size of the text that’s being passed back and forth.

In [ ]:
# ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose= True
)

In [ ]:
conversation.predict(input="Hi, my name is Vijay")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [ ]:
print(memory.buffer)

In [ ]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

# **ConversationBufferWindowMemory**

Given that most conversations will not need context from several messages before, we also have the option of also using ConversationBufferWindowMemory. With this option we can simply control the buffer with a window of the last k messages.

In [ ]:
# ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose= True
)

In [ ]:
conversation.predict(input="Hi, my name is Vijay")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
memory.buffer

# **ConversationTokenBufferMemory**

ConversationTokenBufferMemory simply maintains the buffer by the token count and does not maintain. There is no summarizing with this approach . In fact, it is similar to ConversationBufferWindowMemory, except instead of flushing based on the last number of messages (k), flushing is based on the number of tokens.

In [ ]:
# ConversationTokenBufferMemory
from langchain.memory import ConversationTokenBufferMemory

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

In [ ]:
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})
memory.save_context({"input": "ML is what?"},
                    {"output": "MachineLearning!"})

In [ ]:
memory.load_memory_variables({})

# **ConversationSummaryMemory**

ConversationSummaryMemory does not keep the entire history in memory like ConversationBufferMemory. Nor does it maintain a window. Rather, the ConversationSummaryMemory continually summarizes the conversation as it’s happening, and maintains it so we have context from the beginning of the conversation.

In [ ]:
# ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})

# **Let's Do an Activity**

## **Objective**

Explore different memory strategies in LangChain for managing conversational context and improving interaction quality.

## **Scenario**

You are developing a conversational AI system that interacts with users on various topics, retaining context across multiple messages. Your goal is to implement and compare different memory strategies offered by LangChain to maintain conversation history and enhance user engagement.

## **Steps**

* Choose Memory Strategy

  * ConversationBufferMemory
  * ConversationBufferWindowMemory
  * ConversationTokenBufferMemory
  * ConversationSummaryMemory

* Implement Conversation Chain
* Interact with the System
* Evaluate